In [1]:
push!(LOAD_PATH, "../src")
using Quaternions
using FixedSizeArrays
using RigidBodyDynamics

In [2]:
mechanism = Mechanism{Float64}("world")
world = root_body(mechanism)

RigidBody: "world"

In [3]:
body1 = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body1")));
joint1 = Joint("joint1", QuaternionFloating());
joint1ToWorld = rand(Transform3D{Float64}, joint1.frameBefore, world.frame)
body1ToJoint1 = rand(Transform3D{Float64}, body1.frame, joint1.frameAfter)
attach!(mechanism, world, joint1, joint1ToWorld, body1, body1ToJoint1);

In [4]:
joint2 = Joint("joint2", Revolute(rand(Vec{3, Float64})));
joint2ToBody1 = rand(Transform3D{Float64}, joint2.frameBefore, body1.frame)
body2frame = CartesianFrame3D("body2")
body2 = RigidBody(rand(SpatialInertia{Float64}, body2frame));
body2ToJoint2 = rand(Transform3D{Float64}, body2frame, joint2.frameAfter)
attach!(mechanism, body1, joint2, joint2ToBody1, body2, body2ToJoint2);

In [5]:
mechanism

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2

In [6]:
x = MechanismState(Float64, mechanism);

In [7]:
rand!(x)
setdirty!(x)
x.q[joint1]

7-element Array{Float64,1}:
  0.906054  
  0.252147  
  0.177682  
 -0.289684  
  0.846689  
  0.102059  
  0.00720833

In [8]:
num_positions(x)

8

In [9]:
num_velocities(x)

7

In [10]:
relative_transform(x, joint2.frameAfter, body1.frame)

Transform3D from "after_joint2" to "body1":
rotation: 3.1136303559097604 rad about [-0.027980016502957488,-0.9826674194501167,-0.1832535441069085], translation: Vec(0.15152642712906217,0.4399117570309057,0.14406832857588947)


In [11]:
center_of_mass(x)

Point3D in "world": Vec(1.3893656037257343,0.18635078554589962,2.0882630959715205)

In [12]:
mass(mechanism)

1.201656751999787

In [13]:
transform(x, Point3D{Float64}(joint2.frameAfter), root_frame(mechanism))

Point3D in "world": Vec(1.5509643569599096,-0.6483434601318783,0.9219028071387598)

In [14]:
relative_twist(x, body2, body1)

Twist of "after_joint2" w.r.t "after_joint1" in "world":
angular: Vec(-0.0459668434584099,0.06382046415283649,-0.02243792639669706), linear: Vec(-0.04428878221717958,-0.007576537934249061,0.06918096280652108)

In [15]:
relative_twist(x, body1, world) + relative_twist(x, body2, body1) + relative_twist(x, world, body2)

Twist of "world" w.r.t "world" in "world":
angular: Vec(0.0,0.0,0.0), linear: Vec(0.0,0.0,0.0)

In [16]:
geometric_jacobian(x, path(mechanism, body2, world))

GeometricJacobian: body: "world", base: "after_joint2", expressed in "world":
6x7 Array{Float64,2}:
  0.317413   0.147     -0.887228   …  -0.0       -0.0       -0.0     
 -0.440697   0.949253   0.250809      -0.0       -0.0       -0.0     
  0.15494    0.278046  -0.387197      -0.0       -0.0       -0.0     
  0.305826  -0.95895   -0.141659       0.147     -0.887228   0.437285
  0.052318  -0.206336  -0.353676       0.949253   0.250809   0.189772
 -0.477713   1.21142    0.0955026  …   0.278046  -0.387197  -0.879073

In [17]:
mass_matrix(x)

7x7 Array{Float64,2}:
  3.31281     -1.25621       1.16847      …   0.566096     -0.776026
 -1.25621      2.32436      -0.79298          0.899738     -0.698849
  1.16847     -0.79298       1.7206          -5.55112e-17  -0.253711
  3.1225e-17   1.03682      -0.566096        -5.55112e-17  -0.530265
 -1.03682      1.66533e-16  -0.899738         0.0           0.465447
  0.566096     0.899738      2.77556e-17  …   1.20166      -0.446799
 -0.776026    -0.698849     -0.253711        -0.446799      0.664739

In [18]:
A = momentum_matrix(x)

MomentumMatrix expressed in "world":
6x7 Array{Float64,2}:
 -2.27914     3.55647   -2.29802   …   0.71608    0.67306   -0.821966 
 -4.26695    -0.375285  -1.67183       1.57995   -2.56496    1.68527  
 -0.474418   -1.01075    1.65231      -0.617413  -0.218912   0.472475 
 -1.16744    -0.545855  -0.715057      1.06614   -0.525466   0.686285 
  0.152615   -1.15495    0.763031     -0.301387  -0.228041   0.471407 
  0.0961859   0.502652  -0.190975  …   0.465278   1.05634   -0.0651111

In [19]:
A * velocity_vector(x)

Momentum expressed in "world":
angular: Vec(1.4588202278743225,-2.2875481188456686,-0.580029423948973), linear: Vec(-0.9798545814832567,-0.7642191062665835,0.4220191212768545)

In [20]:
accel = SpatialAcceleration(joint2.frameAfter, world.frame, joint2.frameAfter, rand(Vec{3, Float64}), rand(Vec{3, Float64}))
transform(x, accel, world.frame)

SpatialAcceleration of "after_joint2" w.r.t "world" in "world":
angular: Vec(-0.9947852336018121,0.6721836611782159,0.20926816392153152), linear: Vec(-1.5622976388458318,-0.5117017354170895,0.8624225406082306)

In [21]:
bias_acceleration(x, body2)

SpatialAcceleration of "after_joint2" w.r.t "after_joint1" in "world":
angular: Vec(-0.0285501506571934,-0.021477126996739287,-0.002599166544938549), linear: Vec(0.016641466075140034,-0.017055940808633062,-0.027012874816883803)

In [22]:
v̇ = [j::Joint => rand(num_velocities(j))::Vector{Float64} for j in joints(mechanism)]
τ = inverse_dynamics(x, v̇)

7-element Array{Float64,1}:
  0.172681
 -3.55947 
  1.49122 
  5.18285 
 -4.91666 
 -9.57363 
 -0.407903

In [23]:
rand_chain_mechanism(Float64, QuaternionFloating, Revolute{Float64}, Revolute{Float64})

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body2, Edge: joint2
      Vertex: body3, Edge: joint3

In [24]:
rand_tree_mechanism(Float64, [QuaternionFloating; [Revolute{Float64} for i = 1 : 10]]...)

Vertex: world (root)
  Vertex: body1, Edge: joint1
    Vertex: body10, Edge: joint10
  Vertex: body2, Edge: joint2
    Vertex: body5, Edge: joint5
      Vertex: body8, Edge: joint8
  Vertex: body3, Edge: joint3
    Vertex: body4, Edge: joint4
      Vertex: body6, Edge: joint6
        Vertex: body7, Edge: joint7
        Vertex: body11, Edge: joint11
  Vertex: body9, Edge: joint9